<a href="https://colab.research.google.com/github/3klyata/hw4/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%94_%D0%97%D0%B0%D0%B2%D0%B4%D0%B0%D0%BD%D0%BD%D1%8F_4_%D0%B4%D0%BE_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%BE%D0%B3%D0%BE_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
from nltk.corpus import wordnet
import nltk
from nltk import pos_tag
import sqlite3

# Завантажуємо необхідні ресурси
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Функція для зчитування тексту з файлу
def read_text_from_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return ""

# Функція для отримання першого визначення слова
def get_first_definition(word):
    synsets = wordnet.synsets(word)
    if synsets:
        return synsets[0].definition()  # Беремо тільки перше визначення
    else:
        return "No definition found"

# Функція для визначення частиномовної належності
def get_pos_tag(word):
    tagged = pos_tag([word])
    pos = tagged[0][1]  # Частиномовний тег (наприклад, 'NN' для іменника)


    if pos.startswith('N'):
        return 'noun'  # Іменник
    elif pos.startswith('V'):
        return 'verb'  # Дієслово
    elif pos.startswith('J'):
        return 'adjective'  # Прикметник
    elif pos.startswith('R'):
        return 'adverb'  # Прислівник
    elif pos == 'IN':
        return 'preposition'  # Прийменник
    elif pos == 'CC':
        return 'conjunction'  # Сполучник
    elif pos == 'PRP' or pos == 'PRP$':
        return 'pronoun'  # Займенник
    elif pos == 'MD':
        return 'modal'  # Модальне дієслово
    else:
        return 'other'

# Функція для перевірки закінчення (чи це множина із закінченням "s", бо в англійській мові немає таких закінчень як у нас)
def get_word_inflection(word, pos_tagging):
    # Для іменників та дієслів перевіряємо на наявність закінчення "s"
    if pos_tagging == 'noun' or pos_tagging == 'verb':
        if word.endswith('s'):
            return 'inflection -s'  # Множина або 3-тя особа однини
        else:
            return 'no inflection'
    return 'not applicable'  # Якщо слово не є іменником чи дієсловом

def main():
    word_details = {}

    # Зчитуємо текст із файлу
    text = read_text_from_file('input.txt')

    # Розділяємо текст на слова
    words = re.findall(r'\b\w+\b', text.lower())
    unique_words = set(words)  # Унікальні слова

    print(f"Words found: {len(words)}, unique words: {len(unique_words)}\n")

    for word in unique_words:
        definition = get_first_definition(word)
        pos_tagging = get_pos_tag(word)
        word_inflection = get_word_inflection(word, pos_tagging)

        # Зберігаємо у словник
        word_details[word] = {
            'definition': definition,
            'pos_tag': pos_tagging,
            'word_inflection': word_inflection
        }

    # Повертаємо словник з результатами
    return word_details

# Викликаємо функцію для отримання даних
word_details = main()

# Створення бази даних
conn = sqlite3.connect("words.db")
cursor = conn.cursor()

# Таблиця words
cursor.execute("""
CREATE TABLE IF NOT EXISTS words (
    word_id INTEGER PRIMARY KEY AUTOINCREMENT,
    meaning TEXT,
    pos_type VARCHAR,
    examples TEXT,
    spelling VARCHAR
)
""")

# Таблиця inflections
cursor.execute("""
CREATE TABLE IF NOT EXISTS inflections (
    word_id INTEGER,
    inflected_form VARCHAR,
    FOREIGN KEY(word_id) REFERENCES words(word_id)
)
""")

# Таблиця pos
cursor.execute("""
CREATE TABLE IF NOT EXISTS pos (
    pos_type VARCHAR PRIMARY KEY
)
""")

# Таблиця inflection_types
cursor.execute("""
CREATE TABLE IF NOT EXISTS inflection_types (
    inflection_type VARCHAR PRIMARY KEY
)
""")

# Для кожного слова з нашого словника word_details
for word, details in word_details.items():
    # Вставляємо основне слово в таблицю 'words'
    cursor.execute("""
        INSERT INTO words (meaning, pos_type, examples, spelling)
        VALUES (?, ?, ?, ?)
    """, (details['definition'], details['pos_tag'], None, word))  # 'examples' залишимо порожнім

    word_id = cursor.lastrowid

    # Вставка інфлексій для кожного слова в таблицю 'inflections'
    word_inflection = details['word_inflection']
    if word_inflection != 'not applicable':
        cursor.execute("""
            INSERT INTO inflections (word_id, inflected_form)
            VALUES (?, ?)
        """, (word_id, word_inflection))

    # Вставка частини мови в таблицю 'pos'
    pos_tag = details['pos_tag']
    cursor.execute("""
        INSERT OR IGNORE INTO pos (pos_type)
        VALUES (?)
    """, (pos_tag,))

    if word_inflection != 'not applicable' and word_inflection != 'no inflection':
        cursor.execute("""
            INSERT OR IGNORE INTO inflection_types (inflection_type)
            VALUES (?)
        """, (word_inflection,))

conn.commit()
conn.close()

print("База даних успішно заповнена!")



Words found: 141, unique words: 51

База даних успішно заповнена!


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
